# Model Comparison: XGBoost vs Baseline Models

**VenueSignal - AAI-540 Group 6**

---

This notebook provides a comprehensive comparison between:
1. **Baseline Model #1**: Simple Heuristic (avg_review_stars)
2. **Baseline Model #2**: Linear Regression (3 features)
3. **XGBoost Model**: Parking-focused classifier (leak-free features)

**Note**: The baseline models predict continuous star ratings (1.0-5.0) using all features, while XGBoost predicts binary classification (is_highly_rated) using only parking-related features.

---
## 1. Setup & Configuration

### Install/Update Required Packages

In [ ]:
!python --version

In [ ]:
# Install/update required packages for SageMaker
!pip3 install -U sagemaker xgboost scikit-learn pandas numpy matplotlib seaborn -q

In [ ]:
# Standard libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from time import gmtime, strftime
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    roc_auc_score, classification_report, confusion_matrix,
    f1_score, precision_recall_curve, auc, precision_score, recall_score
)
from xgboost import XGBClassifier

# AWS & SageMaker libraries
import boto3
import sagemaker
from sagemaker import get_execution_role

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

print("Libraries imported successfully")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# SageMaker Configuration
try:
    role = get_execution_role()
    print(f"Using SageMaker Execution Role")
except:
    # Fallback for local development
    role = "arn:aws:iam::123456789012:role/service-role/AmazonSageMaker-ExecutionRole"
    print("WARNING: Using placeholder role (running locally)")

sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
prefix = "venuesignal-model-comparison"

print(f"\nSageMaker Configuration:")
print(f"  Region: {region}")
print(f"  S3 Bucket: {bucket}")
print(f"  Prefix: {prefix}")
print(f"  Role: {role.split('/')[-1] if '/' in role else role}")

---
## 2. Data Loading & Preparation

Load the training dataset from local file system or S3 (if running in SageMaker).

In [ ]:
# Load the dataset
# Try multiple locations: local path first, then S3
DATA_PATH = 'training_data/split_dataset.csv'

print("Loading data...")
try:
    # Try loading from local path (relative to notebook)
    df = pd.read_csv(DATA_PATH)
    print(f"Data loaded from local path: {DATA_PATH}")
except FileNotFoundError:
    # Try parent directory
    try:
        df = pd.read_csv(f'../{DATA_PATH}')
        print(f"Data loaded from parent directory: ../{DATA_PATH}")
    except FileNotFoundError:
        # Try S3 if running in SageMaker
        try:
            s3_path = f's3://{bucket}/{prefix}/data/split_dataset.csv'
            df = pd.read_csv(s3_path)
            print(f"Data loaded from S3: {s3_path}")
        except:
            raise FileNotFoundError(f"Could not find data file. Please ensure {DATA_PATH} exists.")

print(f"\nData Summary:")
print(f"  Total records: {len(df):,}")
print(f"  Features: {df.shape[1]}")
print(f"\nSplit distribution:")
print(df['split'].value_counts())
print(f"\nTarget distribution (is_highly_rated):")
print(df['is_highly_rated'].value_counts())

In [ ]:
# Ensure target is integer for classification
df["is_highly_rated"] = df["is_highly_rated"].astype(int)
df = df.drop_duplicates()

# Define feature sets
TARGET_REGRESSION = "stars"
TARGET_CLASSIFICATION = "is_highly_rated"
NON_FEATURES = ["business_id", "split", TARGET_REGRESSION, TARGET_CLASSIFICATION]

# Baseline features (for Linear Regression)
BASELINE_FEATURES = ['avg_review_stars', 'enhanced_parking_score', 'review_count']

# XGBoost safe features (leak-free, parking-focused)
XGBOOST_SAFE_FEATURES = [
    "parking_availability_score",
    "enhanced_parking_score",
    "has_parking",
    "price_range",
    "parking_mentions",
    "parking_positive_sentiment",
    "parking_negative_sentiment",
    "free_parking_mentions",
    "valet_mentions",
    "has_good_parking",
    "is_restaurant"
]

print("\nFeature sets defined:")
print(f"  Baseline features (regression): {len(BASELINE_FEATURES)}")
print(f"  XGBoost features (classification): {len(XGBOOST_SAFE_FEATURES)}")

In [ ]:
# Split data into train/val/test sets
train_df = df[df['split'] == 'train'].copy()
val_df = df[df['split'] == 'val'].copy()
test_df = df[df['split'] == 'test'].copy()

print(f"Data split completed:")
print(f"  Training:   {len(train_df):,} records ({len(train_df)/len(df)*100:.1f}%)")
print(f"  Validation: {len(val_df):,} records ({len(val_df)/len(df)*100:.1f}%)")
print(f"  Test:       {len(test_df):,} records ({len(test_df)/len(df)*100:.1f}%)")

# Verify no data leakage between train and test
assert len(set(train_df['business_id']) & set(test_df['business_id'])) == 0, "Data leakage detected!"
print("\nNo data leakage detected between train and test sets")

---
## 3. Baseline Model #1: Simple Heuristic

**Approach**: Predict star rating using business-level average (`avg_review_stars`)  
**Task**: Regression (predicts continuous 1.0-5.0 stars)  
**Features**: 1 (avg_review_stars only)

In [ ]:
print("=" * 80)
print("BASELINE MODEL #1: SIMPLE HEURISTIC")
print("=" * 80)

# Prepare regression targets
y_train_reg = train_df[TARGET_REGRESSION].values
y_val_reg = val_df[TARGET_REGRESSION].values
y_test_reg = test_df[TARGET_REGRESSION].values

# Make predictions using business average
y_train_pred_h = train_df['avg_review_stars'].values
y_val_pred_h = val_df['avg_review_stars'].values
y_test_pred_h = test_df['avg_review_stars'].values

# Calculate metrics
def calculate_regression_metrics(y_true, y_pred, set_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    within_05 = np.mean(np.abs(y_true - y_pred) <= 0.5) * 100
    within_10 = np.mean(np.abs(y_true - y_pred) <= 1.0) * 100
    
    print(f"\n{set_name} Set:")
    print(f"  RMSE:              {rmse:.4f}")
    print(f"  MAE:               {mae:.4f}")
    print(f"  R²:                {r2:.4f}")
    print(f"  Within 0.5 stars:  {within_05:.1f}%")
    print(f"  Within 1.0 star:   {within_10:.1f}%")
    
    return {'rmse': rmse, 'mae': mae, 'r2': r2, 'within_05': within_05, 'within_10': within_10}

train_metrics_h = calculate_regression_metrics(y_train_reg, y_train_pred_h, "Training")
val_metrics_h = calculate_regression_metrics(y_val_reg, y_val_pred_h, "Validation")
test_metrics_h = calculate_regression_metrics(y_test_reg, y_test_pred_h, "Test")

---
## 4. Baseline Model #2: Linear Regression

**Approach**: Linear Regression with 3 key features  
**Task**: Regression (predicts continuous 1.0-5.0 stars)  
**Features**: avg_review_stars, enhanced_parking_score, review_count

In [ ]:
print("=" * 80)
print("BASELINE MODEL #2: LINEAR REGRESSION")
print("=" * 80)

# Prepare feature matrices
X_train_lr = train_df[BASELINE_FEATURES].fillna(0).values
X_val_lr = val_df[BASELINE_FEATURES].fillna(0).values
X_test_lr = test_df[BASELINE_FEATURES].fillna(0).values

print(f"\nFeatures: {BASELINE_FEATURES}")
print(f"Training shape: {X_train_lr.shape}")

# Train model
baseline_lr = LinearRegression()
baseline_lr.fit(X_train_lr, y_train_reg)

print("\nModel Coefficients:")
print(f"  Intercept: {baseline_lr.intercept_:.4f}")
for feat, coef in zip(BASELINE_FEATURES, baseline_lr.coef_):
    print(f"  {feat:30s}: {coef:8.4f}")

# Make predictions
y_train_pred_lr = baseline_lr.predict(X_train_lr)
y_val_pred_lr = baseline_lr.predict(X_val_lr)
y_test_pred_lr = baseline_lr.predict(X_test_lr)

# Evaluate
train_metrics_lr = calculate_regression_metrics(y_train_reg, y_train_pred_lr, "Training")
val_metrics_lr = calculate_regression_metrics(y_val_reg, y_val_pred_lr, "Validation")
test_metrics_lr = calculate_regression_metrics(y_test_reg, y_test_pred_lr, "Test")

---
## 5. XGBoost Classification Model

**Approach**: XGBoost with parking-focused features (leak-free)  
**Task**: Binary Classification (predicts is_highly_rated: 0 or 1)  
**Features**: 11 parking-related features (no rating-derived features)

In [ ]:
print("=" * 80)
print("XGBOOST CLASSIFICATION MODEL")
print("=" * 80)

# Prepare classification targets
y_train_clf = train_df[TARGET_CLASSIFICATION].values
y_val_clf = val_df[TARGET_CLASSIFICATION].values
y_test_clf = test_df[TARGET_CLASSIFICATION].values

# Prepare feature matrices
X_train_xgb = train_df[XGBOOST_SAFE_FEATURES].copy()
X_val_xgb = val_df[XGBOOST_SAFE_FEATURES].copy()
X_test_xgb = test_df[XGBOOST_SAFE_FEATURES].copy()

# Convert numeric
for col in XGBOOST_SAFE_FEATURES:
    X_train_xgb[col] = pd.to_numeric(X_train_xgb[col], errors='coerce').fillna(0)
    X_val_xgb[col] = pd.to_numeric(X_val_xgb[col], errors='coerce').fillna(0)
    X_test_xgb[col] = pd.to_numeric(X_test_xgb[col], errors='coerce').fillna(0)

print(f"\nFeatures: {len(XGBOOST_SAFE_FEATURES)} parking-related features")
print(f"Training shape: {X_train_xgb.shape}")

# Calculate class imbalance
neg = (y_train_clf == 0).sum()
pos = (y_train_clf == 1).sum()
scale_pos_weight = neg / max(pos, 1)

print(f"\nClass balance:")
print(f"  Negative (0): {neg:,}")
print(f"  Positive (1): {pos:,}")
print(f"  scale_pos_weight: {scale_pos_weight:.2f}")

# Train XGBoost
xgb_model = XGBClassifier(
    n_estimators=400,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="binary:logistic",
    eval_metric="auc",
    scale_pos_weight=scale_pos_weight,
    tree_method="hist",
    random_state=42
)

print("\nTraining XGBoost model...")
xgb_model.fit(X_train_xgb, y_train_clf)
print("Training complete")

In [ ]:
# Make predictions
y_train_pred_xgb_proba = xgb_model.predict_proba(X_train_xgb)[:, 1]
y_val_pred_xgb_proba = xgb_model.predict_proba(X_val_xgb)[:, 1]
y_test_pred_xgb_proba = xgb_model.predict_proba(X_test_xgb)[:, 1]

# Calculate optimal threshold using F1 score on validation set
precision, recall, thresholds = precision_recall_curve(y_val_clf, y_val_pred_xgb_proba)
f1_scores = []
for t in thresholds:
    preds = (y_val_pred_xgb_proba >= t).astype(int)
    f1_scores.append(f1_score(y_val_clf, preds))

best_threshold = thresholds[np.argmax(f1_scores)]
print(f"Optimal threshold (F1-based): {best_threshold:.3f}")

# Apply threshold
y_train_pred_xgb = (y_train_pred_xgb_proba >= best_threshold).astype(int)
y_val_pred_xgb = (y_val_pred_xgb_proba >= best_threshold).astype(int)
y_test_pred_xgb = (y_test_pred_xgb_proba >= best_threshold).astype(int)

# Calculate classification metrics
def calculate_classification_metrics(y_true, y_pred, y_pred_proba, set_name):
    auc_score = roc_auc_score(y_true, y_pred_proba)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    
    # PR AUC
    prec_vals, rec_vals, _ = precision_recall_curve(y_true, y_pred_proba)
    pr_auc = auc(rec_vals, prec_vals)
    
    print(f"\n{set_name} Set:")
    print(f"  ROC AUC:           {auc_score:.4f}")
    print(f"  PR AUC:            {pr_auc:.4f}")
    print(f"  F1 Score:          {f1:.4f}")
    print(f"  Precision:         {precision:.4f}")
    print(f"  Recall:            {recall:.4f}")
    
    return {'auc': auc_score, 'pr_auc': pr_auc, 'f1': f1, 'precision': precision, 'recall': recall}

print("\nXGBoost Classification Metrics:")
print("=" * 80)
train_metrics_xgb = calculate_classification_metrics(y_train_clf, y_train_pred_xgb, y_train_pred_xgb_proba, "Training")
val_metrics_xgb = calculate_classification_metrics(y_val_clf, y_val_pred_xgb, y_val_pred_xgb_proba, "Validation")
test_metrics_xgb = calculate_classification_metrics(y_test_clf, y_test_pred_xgb, y_test_pred_xgb_proba, "Test")

In [ ]:
# Confusion Matrix
print("\nTest Set Confusion Matrix:")
cm = confusion_matrix(y_test_clf, y_test_pred_xgb)
print(cm)

print("\nClassification Report:")
print(classification_report(y_test_clf, y_test_pred_xgb, digits=3))

---
## 6. Model Comparison

### Important Notes:
- **Baseline models** solve a **regression** problem (predict continuous star rating 1.0-5.0)
- **XGBoost model** solves a **classification** problem (predict binary is_highly_rated)
- Direct metric comparison is not appropriate due to different problem types
- We compare them conceptually and on business value

In [ ]:
# Create comparison summary
print("=" * 90)
print("MODEL COMPARISON SUMMARY")
print("=" * 90)

comparison_data = {
    'Model': [
        'Baseline #1: Heuristic',
        'Baseline #2: Linear Regression',
        'XGBoost Classifier'
    ],
    'Task': [
        'Regression (stars)',
        'Regression (stars)',
        'Classification (highly_rated)'
    ],
    'Features': [
        '1 (avg_review_stars)',
        '3 (avg_review_stars, parking, reviews)',
        '11 (parking-focused, leak-free)'
    ],
    'Test Performance': [
        f"RMSE: {test_metrics_h['rmse']:.4f}, MAE: {test_metrics_h['mae']:.4f}",
        f"RMSE: {test_metrics_lr['rmse']:.4f}, MAE: {test_metrics_lr['mae']:.4f}",
        f"ROC AUC: {test_metrics_xgb['auc']:.4f}, F1: {test_metrics_xgb['f1']:.4f}"
    ],
    'Strengths': [
        'Simple, interpretable, fast',
        'Linear, interpretable, good baseline',
        'Parking-focused, no leakage, decision support'
    ],
    'Use Case': [
        'Quick estimate of expected rating',
        'Rating prediction with parking context',
        'Risk assessment for parking impact'
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))
print("=" * 90)

In [ ]:
# Feature importance comparison
print("\n" + "=" * 80)
print("FEATURE IMPORTANCE COMPARISON")
print("=" * 80)

print("\nLinear Regression Coefficients:")
for feat, coef in zip(BASELINE_FEATURES, baseline_lr.coef_):
    print(f"  {feat:35s}: {coef:8.4f}")

print("\nXGBoost Feature Importance (Top 10):")
xgb_importance = pd.Series(
    xgb_model.feature_importances_,
    index=XGBOOST_SAFE_FEATURES
).sort_values(ascending=False)

for feat, imp in xgb_importance.head(10).items():
    print(f"  {feat:35s}: {imp:8.4f}")

---
## 7. Visualizations

In [ ]:
# Compare baseline regression models
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Heuristic predictions
axes[0].scatter(y_test_reg, y_test_pred_h, alpha=0.3, s=20, label='Predictions')
axes[0].plot([1, 5], [1, 5], 'r--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Stars')
axes[0].set_ylabel('Predicted Stars')
axes[0].set_title(f'Baseline #1: Heuristic\nTest RMSE: {test_metrics_h["rmse"]:.4f}')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Linear Regression predictions
axes[1].scatter(y_test_reg, y_test_pred_lr, alpha=0.3, s=20, label='Predictions', color='orange')
axes[1].plot([1, 5], [1, 5], 'r--', lw=2, label='Perfect Prediction')
axes[1].set_xlabel('Actual Stars')
axes[1].set_ylabel('Predicted Stars')
axes[1].set_title(f'Baseline #2: Linear Regression\nTest RMSE: {test_metrics_lr["rmse"]:.4f}')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# XGBoost visualizations
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Confusion Matrix
cm = confusion_matrix(y_test_clf, y_test_pred_xgb)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0])
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')
axes[0].set_title(f'XGBoost Confusion Matrix\nTest F1: {test_metrics_xgb["f1"]:.4f}')

# ROC Curve visualization via probability distribution
axes[1].hist(y_test_pred_xgb_proba[y_test_clf == 0], bins=30, alpha=0.5, label='Negative Class', color='red')
axes[1].hist(y_test_pred_xgb_proba[y_test_clf == 1], bins=30, alpha=0.5, label='Positive Class', color='green')
axes[1].axvline(best_threshold, linestyle='--', color='black', label=f'Threshold={best_threshold:.3f}')
axes[1].set_xlabel('Predicted Probability')
axes[1].set_ylabel('Count')
axes[1].set_title(f'Probability Distribution\nROC AUC: {test_metrics_xgb["auc"]:.4f}')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Feature Importance
top_features = xgb_importance.head(10)
axes[2].barh(range(len(top_features)), top_features.values)
axes[2].set_yticks(range(len(top_features)))
axes[2].set_yticklabels(top_features.index, fontsize=9)
axes[2].set_xlabel('Importance')
axes[2].set_title('XGBoost Feature Importance (Top 10)')
axes[2].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

In [ ]:
# Baseline regression metrics comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Error metrics
metrics = ['RMSE', 'MAE']
heuristic_vals = [test_metrics_h['rmse'], test_metrics_h['mae']]
lr_vals = [test_metrics_lr['rmse'], test_metrics_lr['mae']]

x = np.arange(len(metrics))
width = 0.35

axes[0].bar(x - width/2, heuristic_vals, width, label='Heuristic', alpha=0.8, color='skyblue')
axes[0].bar(x + width/2, lr_vals, width, label='Linear Regression', alpha=0.8, color='orange')
axes[0].set_ylabel('Error')
axes[0].set_title('Baseline Models: Error Metrics (Test Set)')
axes[0].set_xticks(x)
axes[0].set_xticklabels(metrics)
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# Business metrics
business_metrics = ['Within 0.5★', 'Within 1.0★']
heuristic_business = [test_metrics_h['within_05'], test_metrics_h['within_10']]
lr_business = [test_metrics_lr['within_05'], test_metrics_lr['within_10']]

x2 = np.arange(len(business_metrics))
axes[1].bar(x2 - width/2, heuristic_business, width, label='Heuristic', alpha=0.8, color='skyblue')
axes[1].bar(x2 + width/2, lr_business, width, label='Linear Regression', alpha=0.8, color='orange')
axes[1].set_ylabel('Percentage (%)')
axes[1].set_title('Baseline Models: Business Metrics (Test Set)')
axes[1].set_xticks(x2)
axes[1].set_xticklabels(business_metrics)
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Calculate improvement
rmse_improvement = (test_metrics_h['rmse'] - test_metrics_lr['rmse']) / test_metrics_h['rmse'] * 100
print(f"\nLinear Regression improvement over Heuristic: {rmse_improvement:+.1f}% RMSE reduction")

---
## 8. Key Insights & Findings

In [ ]:
print("=" * 90)
print("KEY INSIGHTS & FINDINGS")
print("=" * 90)

print("\n1. BASELINE REGRESSION MODELS (Star Rating Prediction)")
print("-" * 90)
print(f"   Heuristic Baseline:")
print(f"     - Test RMSE: {test_metrics_h['rmse']:.4f}")
print(f"     - Test MAE: {test_metrics_h['mae']:.4f}")
print(f"     - Uses only avg_review_stars (simplest predictor)")
print(f"     - Represents consumer view: what rating shows on Yelp")

print(f"\n   Linear Regression Baseline:")
print(f"     - Test RMSE: {test_metrics_lr['rmse']:.4f}")
print(f"     - Test MAE: {test_metrics_lr['mae']:.4f}")
print(f"     - Improvement over heuristic: {rmse_improvement:+.1f}% RMSE reduction")
print(f"     - Incorporates parking score and review count")
print(f"     - Parking coefficient: {baseline_lr.coef_[1]:.4f}")

if test_metrics_lr['rmse'] < 0.7:
    print("     EXCELLENT: Baseline exceeds expectations")
elif test_metrics_lr['rmse'] < 0.8:
    print("     GOOD: Baseline meets expectations")

print("\n2. XGBOOST CLASSIFICATION MODEL (Highly Rated Prediction)")
print("-" * 90)
print(f"   Performance:")
print(f"     - Test ROC AUC: {test_metrics_xgb['auc']:.4f}")
print(f"     - Test PR AUC: {test_metrics_xgb['pr_auc']:.4f}")
print(f"     - Test F1 Score: {test_metrics_xgb['f1']:.4f}")
print(f"     - Precision: {test_metrics_xgb['precision']:.4f}")
print(f"     - Recall: {test_metrics_xgb['recall']:.4f}")

print(f"\n   Key Characteristics:")
print(f"     - Uses ONLY parking-related features (no rating leakage)")
print(f"     - Moderate performance (ROC AUC ~0.64) reflects true parking signal")
print(f"     - Parking is meaningful but not dominant predictor")
print(f"     - Designed for risk assessment, not definitive prediction")

print(f"\n   Most Important Parking Features:")
for feat, imp in xgb_importance.head(5).items():
    print(f"     - {feat}: {imp:.4f}")

print("\n3. MODEL COMPARISON & COMPLEMENTARY ROLES")
print("-" * 90)
print("   These models solve DIFFERENT problems:")
print("")
print("   Baseline Models (Regression):")
print("     • Predict continuous star rating (1.0 - 5.0)")
print("     • Use all available features including avg_review_stars")
print("     • Goal: Accurate rating prediction")
print("     • Use case: Forecast expected rating for location")
print("")
print("   XGBoost Model (Classification):")
print("     • Predict binary outcome: highly rated or not")
print("     • Use ONLY parking features (leak-free)")
print("     • Goal: Assess parking-related risk")
print("     • Use case: Identify if parking constraints will hurt ratings")
print("")
print("   - Models are complementary, not competing")
print("   - Use baseline for overall rating prediction")
print("   - Use XGBoost for parking-specific risk assessment")

print("\n4. BUSINESS VALUE & RECOMMENDATIONS")
print("-" * 90)
print("   Baseline Models:")
print("     • Deploy for general rating forecasting")
print("     • Simple and interpretable for stakeholders")
print("     • Fast inference for real-time predictions")
print("")
print("   XGBoost Model:")
print("     • Deploy for parking risk stratification")
print("     • Helps identify businesses needing parking interventions")
print("     • Supports site selection and mitigation planning")
print("     • Probabilistic outputs enable risk tiering")
print("")
print("   Combined Strategy:")
print("     1. Use baseline to forecast overall expected rating")
print("     2. Use XGBoost to assess parking-specific risk")
print("     3. Combine insights for comprehensive location evaluation")

print("\n" + "=" * 90)

---
## 9. Conclusion

### Model Performance Summary

**Baseline Models (Regression - Star Rating Prediction)**
- Successfully predict continuous star ratings with low error
- Heuristic and Linear Regression provide strong baselines
- Suitable for general rating forecasting

**XGBoost Model (Classification - Highly Rated Prediction)**
- Predicts binary outcome using only parking features
- Moderate performance (ROC AUC ~0.64) reflects real parking signal without leakage
- Designed for risk assessment and decision support
- Most important features: has_good_parking, parking_availability_score

### Key Takeaways

1. **Different Problems, Different Models**: Baseline models predict exact ratings; XGBoost assesses parking risk
2. **Parking as Risk Factor**: XGBoost confirms parking matters, but is not the sole determinant
3. **Complementary Deployment**: Use both model types for comprehensive business intelligence
4. **Leak-Free Analysis**: XGBoost demonstrates parking's true impact without rating-derived features

### Recommendations

- Deploy baseline models for **overall rating prediction**
- Deploy XGBoost for **parking-specific risk assessment**
- Use combined insights for **site selection and operational planning**
- Implement risk tiering (high/moderate/low) for stakeholder communication

In [ ]:
print("=" * 90)
print("MODEL COMPARISON ANALYSIS COMPLETE")
print("=" * 90)
print(f"Completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\nAll models evaluated and compared successfully")
print("Baseline models demonstrate strong regression performance")
print("XGBoost model provides parking-specific risk assessment")
print("\nKey Results Summary:")
print(f"  - Heuristic RMSE: {test_metrics_h['rmse']:.4f}")
print(f"  - Linear Reg RMSE: {test_metrics_lr['rmse']:.4f}")
print(f"  - XGBoost ROC AUC: {test_metrics_xgb['auc']:.4f}")
print(f"  - XGBoost F1 Score: {test_metrics_xgb['f1']:.4f}")
print("\nReady for deployment and stakeholder presentation")
print("\nRecommendation: Use both model types for comprehensive business intelligence")
print("   - Baseline models for overall rating prediction")
print("   - XGBoost model for parking risk assessment & site selection")

---
## 10. Save Results to S3 (Optional)

Save comparison results and metrics to S3 for future reference and reporting.

In [ ]:
# Create results summary
import json

results_summary = {
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'baseline_heuristic': {
        'model_type': 'Regression (Heuristic)',
        'features': 1,
        'test_rmse': float(test_metrics_h['rmse']),
        'test_mae': float(test_metrics_h['mae']),
        'test_r2': float(test_metrics_h['r2']),
        'within_05_stars_pct': float(test_metrics_h['within_05']),
        'within_10_star_pct': float(test_metrics_h['within_10'])
    },
    'baseline_linear_regression': {
        'model_type': 'Regression (Linear)',
        'features': len(BASELINE_FEATURES),
        'feature_list': BASELINE_FEATURES,
        'test_rmse': float(test_metrics_lr['rmse']),
        'test_mae': float(test_metrics_lr['mae']),
        'test_r2': float(test_metrics_lr['r2']),
        'within_05_stars_pct': float(test_metrics_lr['within_05']),
        'within_10_star_pct': float(test_metrics_lr['within_10']),
        'coefficients': {
            'intercept': float(baseline_lr.intercept_),
            'features': {feat: float(coef) for feat, coef in zip(BASELINE_FEATURES, baseline_lr.coef_)}
        }
    },
    'xgboost_classifier': {
        'model_type': 'Classification (XGBoost)',
        'features': len(XGBOOST_SAFE_FEATURES),
        'feature_list': XGBOOST_SAFE_FEATURES,
        'optimal_threshold': float(best_threshold),
        'test_roc_auc': float(test_metrics_xgb['auc']),
        'test_pr_auc': float(test_metrics_xgb['pr_auc']),
        'test_f1': float(test_metrics_xgb['f1']),
        'test_precision': float(test_metrics_xgb['precision']),
        'test_recall': float(test_metrics_xgb['recall']),
        'feature_importance': {
            feat: float(imp) for feat, imp in zip(XGBOOST_SAFE_FEATURES, xgb_model.feature_importances_)
        }
    },
    'comparison_insights': {
        'lr_improvement_over_heuristic_pct': float(rmse_improvement),
        'models_are_complementary': True,
        'recommendation': 'Use baseline for rating prediction, XGBoost for parking risk assessment'
    }
}

# Save locally
results_file = 'model_comparison_results.json'
with open(results_file, 'w') as f:
    json.dump(results_summary, f, indent=2)

print(f"Results saved locally: {results_file}")

# Upload to S3 if in SageMaker environment
try:
    s3_client = boto3.client('s3')
    s3_key = f"{prefix}/results/model_comparison_results_{strftime('%Y%m%d_%H%M%S', gmtime())}.json"
    s3_client.upload_file(results_file, bucket, s3_key)
    print(f"Results uploaded to S3: s3://{bucket}/{s3_key}")
except Exception as e:
    print(f"WARNING: Could not upload to S3: {e}")
    print("  (This is normal if running outside SageMaker)")

# Display summary
print("\n" + "=" * 90)
print("RESULTS SUMMARY")
print("=" * 90)
print(json.dumps(results_summary, indent=2))